In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models  import Sequential
import warnings
warnings.filterwarnings('ignore')

In [ ]:
genres = 'Hayden Mozart Scarlatti'.split()
dataset_name = 'composers_dataset.csv'

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
for g in genres:
    img_folder = f'./pgdata/img_data/{g}'
    pathlib.Path(img_folder).mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./pgdata/pieces/{g}'):
        songname = f'./pgdata/pieces/{g}/{filename}'
        print(songname)
        y, sr = librosa.load(songname, mono=True, duration=30)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'{img_folder}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()
header

In [ ]:
file = open(f'./pgdata/{dataset_name}', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'./pgdata/pieces/{g}'):
        songname = f'./pgdata/pieces/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = []
        to_append.append(filename)
        to_append.append(f'{np.mean(chroma_stft)}')
        to_append.append(f'{np.mean(rmse)}')
        to_append.append(f'{np.mean(spec_cent)}')
        to_append.append(f'{np.mean(spec_bw)}')
        to_append.append(f'{np.mean(rolloff)}')
        to_append.append(f'{np.mean(zcr)}')
        for e in mfcc:
            to_append.append(f'{np.mean(e)}')
        to_append.append(g)
        file = open(f'./pgdata/{dataset_name}', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append)

In [ ]:
data = pd.read_csv(f'./pgdata/{dataset_name}')
data.head()
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
classifier = model.fit(X_train,
                       y_train,
                       epochs=100,
                       batch_size=1)
model.save('./pgdata/composers_model')
model.evaluate(X_test, y_test)